## Simulation: Cortical Gamma Oscillations in Schizophrenia
This simulation models a local cortical circuit (a small patch of brain tissue) to investigate the biological causes of Schizophrenia.
Specifically, it is designed to test the "NMDA Receptor Hypofunction Hypothesis," which suggests that schizophrenia symptoms arise because specific glutamate receptors (NMDA) are underactive, particularly on inhibitory neurons.

The network consists of two neuronal populations that generate a PING (Pyramidal–Interneuron Network Gamma) rhythm.
- Pyramidal Cells (E-cells)Excitatory neurons ($N_E = 200$). These cells provide the recurrent excitation and drive the network dynamics.
- Fast-Spiking Interneurons (I-cells)Inhibitory parvalbumin-positive ($PV^+$) neurons ($N_I = 50$). These neurons act as the pacemaker of the circuit, firing rapidly to synchronize pyramidal-cell activity.
- Connectivity is sparse and random, defined by the connection probabilities$$P_{EE},\; P_{EI},\; P_{IE},\; P_{II}, $$corresponding to excitatory–excitatory, excitatory–inhibitory, inhibitory–excitatory, and inhibitory–inhibitory synapses.

All neurons are described by quadratic integrate-and-fire (QIF) dynamics, which are mathematically equivalent to the Izhikevich model near spike initiation and provide a more realistic description of excitability than leaky integrate-and-fire models.The membrane potential evolves according to$$C \frac{dV}{dt} = g_L \frac{(V - E_L)(V - V_T)}{V_T - E_L} - I_{\text{syn}} - I_{\text{adapt}} + I_{\text{ext}} + I_{\text{noise}} .$$Intrinsic ParametersEach neuron is characterized by:$$C,\; g_L,\; E_L,\; V_T,\; V_R .$$A key biological asymmetry is the difference in leak conductance:$$g_L^{(I)} = 0.5 \;\text{mS/cm}^2, \qquad g_L^{(E)} = 0.05 \;\text{mS/cm}^2,$$making interneurons significantly less excitable at rest and strongly dependent on synaptic input

Pyramidal neurons include a slow spike-frequency adaptation variable $z$:$$\frac{dz}{dt} = -a z,$$with an increment at each spike,$$z \leftarrow z + d .$$This mechanism limits excessive firing and stabilizes gamma oscillations. Interneurons do not exhibit adaptation

Synaptic DynamicsSynaptic transmission is mediated by AMPA, NMDA, and GABA$_A$ receptors.AMPA and GABA SynapsesFast excitatory (AMPA) and inhibitory (GABA) currents are modeled as linear conductances:$$I = g \cdot s \cdot (V - E_{\text{rev}}),$$where $g$ is the maximal conductance, $s$ the synaptic gating variable, and $E_{\text{rev}}$ the reversal potential.NMDA Synapses (Voltage-Dependent Magnesium Block)NMDA currents include a voltage-dependent magnesium block following Jahr & Stevens (1990):$$I_{\text{NMDA}} = g_{\text{NMDA}} \cdot s \cdot (V - E_{\text{exc}}) \left( \frac{1}{1 + [Mg^{2+}] \cdot \exp(-0.062 \cdot V / 3.57)} \right).$$At hyperpolarized potentials, $Mg^{2+}$ blocks the NMDA channel; depolarization relieves this block, introducing a strong nonlinearity in synaptic excitation.5. Synaptic ConductancesThe model includes the following synaptic conductances:$$g_{EE},\; g_{NE},\; g_{EI},\; g_{NI},\; g_{IE},\; g_{II},$$corresponding to AMPA, NMDA, and GABA connections between excitatory and inhibitory populations. Synaptic time constants and reversal potentials are receptor-specific and fixed.


To avoid contamination from initialization artifacts, firing rates were computed after discarding an initial transient period (first 200 ms). This ensures that all reported quantities reflect steady-state network dynamics rather than nonstationary behavior induced by arbitrary initial conditions.


In [9]:
import importlib
import model
importlib.reload(model)
from model.model import network
from model.model import UA_PER_CM2_TO_SI, N_E

## Implementation tuning 




Before reproducing the experiments in the paper, we performed a limited amount of **implementation tuning**. This tuning was **not part of the scientific results**, but was necessary to obtain a stable and numerically correct realization of the model.

In particular, we:
- verified **numerical convergence** with respect to the integration timestep (`dt`) and fixed it once firing rates were stable,
- introduced a single **synaptic scaling factor** (`alpha_EI`) to resolve an underspecified discretization detail (normalization of E→I synaptic gating increments), and selected a value that produced a stable excitatory–inhibitory balance,
- fixed all other model parameters exactly as specified in the paper.

These steps define a **single, fixed model instantiation** and do not correspond to parameter fitting or experimental manipulation.

After this implementation tuning was completed, all subsequent simulations follow the paper’s methodology and vary only **experimental control parameters**, such as external input to excitatory neurons and NMDA conductance onto inhibitory neurons (`gNI`), as described in the original study.

In [2]:
for alpha_EI in [0.30, 0.20, 0.15, 0.10]:
    out=network(alpha_EI=alpha_EI, dt=0.2, rng_seed=1)
    print(out)

WARNING    Cannot use Cython, a test compilation failed: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/ (DistutilsPlatformError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]


{'alpha_EI': 0.3, 'dt': 200.0, 'rate_E_ss': 45.425, 'rate_I_ss': 141.05555555555554, 'I_app_E': 0.04, 'I_app_I': 0.0, 'vmax0_mV': 19.959983513459797, 'spE_t': array([0.0134, 0.0134, 0.0134, ..., 1.9992, 1.9992, 1.9992],
      shape=(18516,)), 'spI_t': array([0.0164, 0.0164, 0.0166, ..., 1.9998, 1.9998, 1.9998],
      shape=(13884,))}
{'alpha_EI': 0.2, 'dt': 200.0, 'rate_E_ss': 55.263888888888886, 'rate_I_ss': 107.17777777777778, 'I_app_E': 0.04, 'I_app_I': 0.0, 'vmax0_mV': 19.956900649597262, 'spE_t': array([0.0134, 0.0134, 0.0134, ..., 1.9998, 1.9998, 1.9998],
      shape=(22427,)), 'spI_t': array([0.0172, 0.0174, 0.0174, ..., 1.9994, 1.9996, 1.9998],
      shape=(10539,))}
{'alpha_EI': 0.15, 'dt': 200.0, 'rate_E_ss': 62.875, 'rate_I_ss': 80.52222222222221, 'I_app_E': 0.04, 'I_app_I': 0.0, 'vmax0_mV': 19.99164309824145, 'spE_t': array([0.0134, 0.0134, 0.0134, ..., 1.9996, 1.9996, 1.9998],
      shape=(25438,)), 'spI_t': array([0.0184, 0.0186, 0.0188, ..., 1.9994, 1.9994, 1.9998], shap

We selected **`alpha_EI = 0.15`** because it lies in the interior of a balanced regime in which:
- both excitatory and inhibitory populations are active,
- inhibitory firing rates remain higher than excitatory firing rates,
- firing rates are not extreme,
- and the qualitative network behavior is robust to small parameter changes.




In [4]:
for dt in [0.02, 0.01, 0.005]:
    out = network(alpha_EI=0.15, dt=dt, rng_seed=1)
    print(out)

{'alpha_EI': 0.15, 'dt': 20.0, 'rate_E_ss': 63.02222222222222, 'rate_I_ss': 82.82222222222222, 'I_app_E': 0.04, 'I_app_I': 0.0, 'vmax0_mV': 19.99879770407278}
{'alpha_EI': 0.15, 'dt': 10.0, 'rate_E_ss': 63.019444444444446, 'rate_I_ss': 83.0, 'I_app_E': 0.04, 'I_app_I': 0.0, 'vmax0_mV': 19.99810280804032}
{'alpha_EI': 0.15, 'dt': 5.0, 'rate_E_ss': 63.03333333333333, 'rate_I_ss': 83.11111111111111, 'I_app_E': 0.04, 'I_app_I': 0.0, 'vmax0_mV': 19.99979188145056}


We selected **`dt = 0.02 ms`** because it lies in the interior of a numerically converged regime in which:
- steady-state firing rates of both excitatory and inhibitory populations are stable under timestep refinement,
- halving the timestep produces negligible changes in firing rates (≪ 1%),
- the qualitative dynamical regime of the network is preserved,
- and the timestep is sufficiently small to accurately resolve fast spike dynamics without unnecessary computational cost.


### Choice of the spike cutoff voltage (`V_peak`)

The spike cutoff voltage `V_peak` is a numerical parameter used to terminate the divergent trajectory of the quadratic integrate-and-fire model. It does not correspond to a biophysical action-potential peak and should therefore be chosen such that network observables are insensitive to its precise value.

We tested several values of `V_peak` (0 mV, 10 mV, and 20 mV) while keeping all other parameters fixed. Lower cutoff values led to systematically higher firing rates due to earlier spike detection in the discrete-time integration scheme.

We selected **`V_peak = 20 mV`** because it lies in a regime where firing rates are lowest and least sensitive to the precise cutoff value, providing the best approximation to the ideal infinite-voltage spike definition of the QIF model. All subsequent simulations fix `V_peak` at this value.


# 1. NMDA Conductance onto Fast-Spiking Interneurons

Parameter $g_{NI}$: NMDA synaptic conductance onto inhibitory (FSI) neurons.

How it is varied
- The parameter is swept over a wide range, including:
    - $g_{NI} = 0$ (complete NMDA removal)
    - Intermediate values (e.g., $0.006$–$0.03$)
    - High values (e.g., $0.054$)
    
Purpose: 
- To model NMDA hypofunction versus hyperfunction on FSIs.
- To test for non-monotonic (inverted-U) effects on gamma power

In [ ]:
I_app_I = 0.5 * UA_PER_CM2_TO_SI 

for g in [0.0, 0.006, 0.012, 0.018, 0.024, 0.054]:
    
    out = network(alpha_EI=0.15, dt=0.02, rng_seed=4, gNI_mS_cm2=g)
    print(g, out["rate_E_ss"], out["rate_I_ss"], out["I_app_I"])

0.0 85.10277777777777 7.055555555555555 0.005
0.006 64.89722222222221 76.12222222222222 0.005
0.012 53.55277777777778 114.7 0.005
0.018 46.02777777777777 140.04444444444442 0.005
0.024 40.519444444444446 158.13333333333333 0.005
0.054 25.819444444444443 204.82222222222222 0.005


### Effect of NMDA conductance onto inhibitory neurons (`gNI`)

Increasing the NMDA conductance from excitatory to inhibitory neurons (`gNI`) produces a systematic shift in network balance. As `gNI` increases, inhibitory neurons receive stronger and more persistent excitation, leading to a marked increase in inhibitory firing rates. The resulting feedback inhibition progressively suppresses excitatory firing.

For very small `gNI`, inhibition is weak and excitatory neurons dominate the network dynamics. For large `gNI`, inhibition becomes overly strong and suppresses excitatory activity. The most relevant dynamical regime lies at intermediate `gNI`, where both populations are active and inhibitory firing rates exceed excitatory rates. This regime corresponds to the parameter range in which the paper reports robust gamma-band oscillations.
